# Installation Notes for K8s and KF Locally on M1

## Docker
https://docs.docker.com/docker-for-mac/install/
```
# You must install Rosetta 2 as some binaries are still Darwin/AMD64
$ softwareupdate --install-rosetta

# Download Docker Desktop installer for Mac with Apple silicon (it contains all other Docker tools)

# Choose "Use the new Virtualization framework" in the Experimental Features
```

## Docker / Kubernetes
https://andrewlock.net/running-kubernetes-and-the-dashboard-with-docker-desktop/
```
# Step 1: Manually enable Kubernetes in Docker desktop preferences

# Check it K8s cluster is running
$ kubectl version

# Check the state of your Docker Desktop cluster
$ kubectl get nodes

$ docker container ls # should show no K8s containers

$ docker info # shows many K8s containers working

# See K8s pods
$ kubectl get pods -n kube-system

# Install Kubernetes Dashboard
# https://kubernetes.io/docs/tasks/access-application-cluster/web-ui-dashboard/
$ kubectl apply -f https://raw.githubusercontent.com/kubernetes/dashboard/v2.2.0/aio/deploy/recommended.yaml

# Disabling the login prompt in Kubernetes Dashboard
$ kubectl patch deployment kubernetes-dashboard -n kubernetes-dashboard --type 'json' -p '[{"op": "add", "path": "/spec/template/spec/containers/0/args/-", "value": "--enable-skip-login"}]'

# Viewing the Kubernetes Dashboard
$ kubectl proxy
# Open: http://localhost:8001/api/v1/namespaces/kubernetes-dashboard/services/https:kubernetes-dashboard:/proxy/

$ kubectl cluster-info
$ kubectl get service

# If the address is already in use:
$ ps aux | grep kubectl
$ kill -9 11803

# Delete Dashboard
#$ kubectl delete -f https://raw.githubusercontent.com/kubernetes/dashboard/v2.2.0/aio/deploy/recommended.yaml

# Installing Kubernetes Metrics Server
$ kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/download/v0.4.2/components.yaml
$ kubectl patch deployment metrics-server -n kube-system --type 'json' -p '[{"op": "add", "path": "/spec/template/spec/containers/0/args/-", "value": "--kubelet-insecure-tls"}]'

# Delete Metrics Server
#$ kubectl delete -f https://github.com/kubernetes-sigs/metrics-server/releases/download/v0.4.2/components.yaml
```

## HashiCorp Vagrant
A tool for building and managing (development and rather local) virtual machine environments in a single workflow.

Download and install a binary package: https://learn.hashicorp.com/tutorials/vagrant/getting-started-install?in=vagrant/getting-started

## Kubeflow on macOS
https://v0-6.kubeflow.org/docs/started/workstation/getting-started-macos/

## MiniKF
https://v0-6.kubeflow.org/docs/other-guides/virtual-dev/getting-started-minikf/
 

!!! VirtualBox does nt support M1

```
# Install Vagrant to manage VMs
# https://www.vagrantup.com/downloads
$ brew tap hashicorp/tap
$ brew install vagrant

#Install VirtualBox: https://www.virtualbox.org/wiki/Downloads

# MiniKF installation
$ vagrant init arrikto/minikf
$ vagrant up

# MiniKF will take a few minutes to boot. When this is done, navigate to http://10.10.10.10 and follow the on-screen instructions to start Kubeflow and Rok.

# Upgrade the MiniKF box to the latest version
$ vagrant box update

# Ensure you have updated to the latest version
$ vagrant box list

# Upgrade the vagrant-persistent-storage plugin to v0.0.47 or later
$ vagrant plugin update vagrant-persistent-storage

# Destroy the VM
$ vagrant destroy

# Remove all local state (all of your customization in MiniKF)
$ rm minikf-user-data.vdi

# Re-create your VM
$ vagrant up

!!! VirtualBox does nt support M1
```

## Minikube
Minikube runs a simple, single-node Kubernetes cluster inside a virtual machine (VM)

- Minikube for Kubeflow: https://v0-6.kubeflow.org/docs/other-guides/virtual-dev/getting-started-minikube/
- !!! Minikube start: https://minikube.sigs.k8s.io/docs/start/
- Deploying Kubeflow on Existing Clusters: https://v0-6.kubeflow.org/docs/started/k8s/overview/

```
$ curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-darwin-arm64
$ sudo install minikube-darwin-arm64 /usr/local/bin/minikube

$ minikube start # (selected Docker driver)
#$ minikube start --cpus 4 --memory 8g --disk-size 60g
$ minikube start --cpus 4 --memory 8096 --disk-size=40g
#$ minikube start --cpus 4 --memory 8g --disk-size 60g --driver=vmware # not supported
#$ minikube start --cpus 4 --memory 8g --disk-size 60g --driver=virtualbox # not supported
#$ minikube start --cpus 4 --memory 8g --disk-size 60g --driver=parallels # not supported
# kubectl is now configured to use "minikube" cluster and "default" namespace by default

# Access your shiny new cluster
$ kubectl get po -A

# Download the appropriate version of kubectl (if needed)
$ minikube kubectl -- get po -A

# Open minikube dashboard
$ minikube dashboard


# Create and run a sample deployment and expose it on port 8080
$ kubectl create deployment hello-minikube --image=k8s.gcr.io/echoserver:1.4
$ kubectl expose deployment hello-minikube --type=NodePort --port=8080
$ kubectl get services hello-minikube

# If not working see below
$ minikube service hello-minikube

# Alternatively, use kubectl to forward the port
kubectl port-forward service/hello-minikube 7080:8080
# Your application is now available at http://localhost:7080/

$ minikube pause
$ minikube unpause
$ minikube stop
$ minikube delete
```

## Kubeflow
- https://v0-6.kubeflow.org/docs/started/k8s/overview/
- https://v0-6.kubeflow.org/docs/started/k8s/kfctl-k8s-istio/
- https://www.kubeflow.org/docs/distributions/azure/deploy/install-kubeflow/

```
# Download the latest configuration
# https://github.com/kubeflow/kubeflow/releases
# https://github.com/kubeflow/kfctl/releases/tag/v1.2.0

$ tar -zvxf ~/MLOps/kfctl_v1.2.0-0-gbc038f9_darwin.tar.gz

# Set up and deploy Kubeflow/KF

# Add kfctl to PATH, to make the kfctl binary easier to use.
$ export PATH=$PATH:~/MLOps

# The following configuration installs Istio by default. Comment out 
# the Istio components in the config file to skip Istio installation. 
# See https://github.com/kubeflow/kubeflow/pull/3663
# Deployment manifest UI
$ export CONFIG_URI="https://raw.githubusercontent.com/kubeflow/manifests/v1.2-branch/kfdef/kfctl_k8s_istio.v1.2.0.yaml"

$ cd ~/MLOps

# Name of your Kubeflow deployment
$ mkdir -p 'kf'
$ cd kf

# Deploy
$ kfctl apply -V -f ${CONFIG_URI}

# Check that the resources have been deployed correctly in namespace kubeflow
$ kubectl get all -n kubeflow

# Check the cluster creation status
kubectl get pods -n kubeflow

# Run KF Central Dashboard
$ export NAMESPACE=istio-system
$ kubectl port-forward -n ${NAMESPACE} svc/istion-ingressgateway 8585:80
# Open http://localhost:8585


# Delete KF
cd ~/MLOps/kf
# If you want to delete all the resources, run:
#kfctl delete -V
```